# python模块:argparse 学习笔记

In [2]:
import argparse
parser = argparse.ArgumentParser(prog='argparse101', description='learn argparse.')

核心函数: argparse.add_argument():
#### ArgumentParser.add_argument(name or flags...[, action][, nargs][, const][, default][, type][, choices][, required][, help][, metavar][, dest])

## 1. name or flags

功能:提供命令行参数中位置参数或可选参数(或关键字参数)的参数名

### 1.1 可选参数

```python
parser.add_argument('-f') # 变量名/返回对象属性名为f
parser.add_argument('-nf') # 变量名/返回对象属性名为nf
parser.add_argument('--foo') # 变量名为foo
parser.add_argument('-f', '--foo') # 变量名为foo
```

### 1.2 位置参数

```python
parser.add_argument('bar') # 变量名为bar
```

### 1.3 示例

In [7]:
parser = argparse.ArgumentParser(prog='PROG')
parser.add_argument('-f', '--foo')
parser.add_argument('bar');

In [8]:
# 在上面的配置下, 只给位置参数而不指定可选参数, 则可选参数变量为none.
parser.parse_args(['BAR'])

Namespace(bar='BAR', foo=None)

In [10]:
parser.parse_args(['BAR', '-foo', 'FOO']) # 指定可选参数

Namespace(bar='BAR', foo='FOO')

In [6]:
parser.parse_args(['--foo', 'FOO']) # 若没有给位置参数, 则报错

usage: PROG [-h] [-f FOO] bar
PROG: error: the following arguments are required: bar


SystemExit: 2

/home/liuxing/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3449: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


## 2. action

功能:指定这个命令行参数要干的事情/动作.

### 2.1 'store'

默认动作, 存储参数的值.

In [60]:
parser = argparse.ArgumentParser()
parser.add_argument('--foo');

In [61]:
parser.parse_args('--foo 1'.split())

Namespace(foo='1')

### 2.2 'store_const'

需要与`const`命名参数组合使用, 存储被`const`命名参数指定的值. 如果命令行指定了该选项, 则变量值为`const`的指定的值, 否则为`None`. 该选项不接受实参.

In [298]:
parser = argparse.ArgumentParser()
parser.add_argument('--foo', action='store_const', const=42);

In [299]:
parser.parse_args('--foo'.split())

Namespace(foo=42)

In [300]:
parser.parse_args(''.split())

Namespace(foo=None)

### 2.3 'store_true'/'store_false'

存储布尔值, 如果命令行不指定该选项, 则默认为`False`/`True`.

In [20]:
parser = argparse.ArgumentParser()
parser.add_argument('--foo', action='store_true')
parser.add_argument('--bar', action='store_false')
parser.add_argument('--baz', action='store_false');

In [1]:
parser.parse_args('--foo --bar'.split()) # 没有指定-baz, 因此对应action不会触发, 值就为True.

NameError: name 'parser' is not defined

### 2.4 'append'

存储一个列表, 多次使用时, 后指定的参数会被追加到该列表.每次使用只能指定一个参数(如果不使用`nargs`命名参数配合的话).

In [151]:
parser = argparse.ArgumentParser()
parser.add_argument('--foo', action='append');

In [152]:
parser.parse_args('--foo 1 --foo 2'.split())

Namespace(foo=['1', '2'])

In [123]:
parser.parse_args('--foo 1 2'.split())

usage: ipykernel_launcher.py [-h] [--foo FOO]
ipykernel_launcher.py: error: unrecognized arguments: 2


SystemExit: 2

/home/liuxing/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3449: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


### 2.5 'append_const'

需要与`const`命名参数组合使用, 存储一个列表, 如果选择了该参数, 则将`const`指定的值追加到列表.

In [91]:
parser = argparse.ArgumentParser()
parser.add_argument('--str', dest='types', action='append_const', const=str)
parser.add_argument('--int', dest='types', action='append_const', const=int);

In [92]:
parser.parse_args('--str --int'.split())

Namespace(types=[<class 'str'>, <class 'int'>])

### 2.6 'count'

计算一个关键字参数出现的次数. 可用于指定详情等级.

In [93]:
parser = argparse.ArgumentParser()
parser.add_argument('--verbose', '-v', action='count', default=0);

In [97]:
parser.parse_args('-vvv'.split())

Namespace(verbose=3)

In [98]:
parser.parse_args('-v -v -v'.split())

Namespace(verbose=3)

### 2.7 'help'

打印所有当前解析器中的选项和参数的完整帮助信息, 然后退出. 默认情况下, 一个 help 动作会被自动加入解析器.

### 2.8 'version'

需要与`version`命名参数配合使用, 打印版本信息并**退出**.

In [102]:
parser = argparse.ArgumentParser(prog='PROG')
parser.add_argument('--version', action='version', version='%(prog)s 2.0');

In [103]:
parser.parse_args(['--version'])

PROG 2.0


SystemExit: 0

### 2.9 'extend'

存储一个列表, 可以一个命令行参数(即选项)一次性指定多个参数值(需要与`nargs`配合), 也可以多次使用. 所有值将存储到一个列表.

In [166]:
parser = argparse.ArgumentParser()
parser.add_argument("--foo", action="extend", nargs="+", type=str);

parser.parse_args("--foo f1 --foo f2 f3".split())

Namespace(foo=['f1', 'f2', 'f3'])

与'append'的区别: 
- 'extend'似乎会把这些单独列表里面的元素拿出来重新组成一个新的列表.
- 'append'似乎会把每次调用时给的参数当作更独立的对象(由`nargs`来决定)来组成列表, 例如下面由于`nargs`指定了"+", 所以每次指定命令行参数时都形成单独的列表, 最后把几个列表当作相互独立的元素组成一个大列表. 例如:

In [135]:
parser = argparse.ArgumentParser()
parser.add_argument("--foo", action="append", nargs="+", type=str);

parser.parse_args("--foo f1 --foo f2 f3".split())

Namespace(foo=[['f1'], ['f2', 'f3']])

## 3. nargs

功能: 关联不同数目的命令行参数到单一动作.

### 3.1 `N`(一个整数)

命令行中的`N`个参数会被聚集到一个列表中, 就是要求命令行参数需要提供指定数量的实参.

In [11]:
parser = argparse.ArgumentParser()
parser.add_argument('--foo', nargs=2) # 比如这里就可以用于要求输入ra dec
parser.add_argument('bar', nargs=1); # 产生一个单元素列表.

In [14]:
parser.parse_args('c --foo a b'.split())

Namespace(bar=['c'], foo=['a', 'b'])

### 3.2 `?`

需要与`const`,`default`命名参数配合使用:
- 如果指定了命令行参数并提供了实参, 则产生一个值为指定实参的**单一项**.
- 如果指定了命令行参数但没有提供实参, 则产生一个值为`const`指定的单一项. 位置参数没有这种情况.
- 如果没有指定命令行参数, 则产生一个值位`default`指定的单一项.

In [3]:
parser = argparse.ArgumentParser()
parser.add_argument('--foo', nargs='?', const='c', default='d')
# 对于位置参数, 要想使用默认值就必须加上 nargs='?'
parser.add_argument('bar', nargs='?', default='d');

In [4]:
parser.print_help()

usage: ipykernel_launcher.py [-h] [--foo [FOO]] [bar]

positional arguments:
  bar

optional arguments:
  -h, --help   show this help message and exit
  --foo [FOO]


In [162]:
# 这里都提供了实参, 所以产生对应值.
parser.parse_args(['XX', '--foo', 'YY'])

Namespace(bar='XX', foo='YY')

In [160]:
# 这里位置参数提供了实参, 而关键字参数指定了但没有提供, 因此关键字参数产生`const`的值.
parser.parse_args(['XX', '--foo'])

Namespace(bar='XX', foo='c')

In [161]:
# 这里都没有提供参数, 因此均产生`default`的值.
parser.parse_args([])

Namespace(bar='d', foo='d')

### 3.3 `*`

给各个命令行参数提供的所有参数被聚集到一个列表, 相当于`nargs=N`时N取无穷大?

In [11]:
parser = argparse.ArgumentParser()
parser.add_argument('--foo', nargs='*', default=None)
parser.add_argument('--bar', nargs='*')
parser.add_argument('baz', nargs='*');

In [7]:
parser.parse_args('a b --foo x y --bar 1 2'.split())

Namespace(bar=['1', '2'], baz=['a', 'b'], foo=['x', 'y'])

In [184]:
# 就算不提供任何实际参数(包括位置参数), 也能产生变量, 值都是空列表.
parser.parse_args('--foo --bar'.split())

Namespace(bar=[], baz=[], foo=[])

### 3.4 `+`

与`*`类似, 给各个命令行参数提供的所有参数被聚集到一个列表. 区别是, 如果是`+`, 则必须提供至少一个实际参数

In [190]:
parser = argparse.ArgumentParser()
parser.add_argument('--foo', nargs='+')
parser.add_argument('--bar', nargs='+')
parser.add_argument('baz', nargs='+');

In [193]:
parser.parse_args('a b --foo x y --bar 1 2'.split())

Namespace(bar=['1', '2'], baz=['a', 'b'], foo=['x', 'y'])

In [ ]:
# 这里--bar参数没有提供实际参数就会报错.
parser.parse_args('a b --foo x --bar'.split())

## 4. const

功能: 保存不从命令行中读取但被各种 ArgumentParser 动作需求的常数值。
最常用的两例为：
- 当 add_argument() 通过 action='store_const' 或 action='append_const' 调用时。这些动作将 const 值添加到 parse_args() 返回的对象的属性中。在 action 的描述中查看实例。
- 当 add_argument() 通过选项（例如 -f 或 --foo）调用并且 nargs='?' 时。这会创建一个可以跟随零个或一个命令行参数的选项。当解析命令行时，如果选项后没有参数，则将用 const 代替。在 nargs 描述中查看实例。

## 5. 默认值

功能: 指定了在命令行参数未出现时应当使用的值。

对于命令行中的关键字参数， default 值在选项未在命令行中出现时使用:

In [203]:
parser = argparse.ArgumentParser()
parser.add_argument('--foo', default=42)
parser.add_argument('bar', nargs='?', default=43);

In [204]:
parser.parse_args([])

Namespace(bar=43, foo=42)

## 6. type

功能: 把提供给命令行参数的实参(字符串类型)转换成其它类型, 也可以是自定义的函数

In [7]:
import pathlib
parser = argparse.ArgumentParser()
parser.add_argument('count', type=int)
parser.add_argument('distance', type=float)
parser.add_argument('street', type=ascii)
parser.add_argument('code_point', type=ord)
parser.add_argument('source_file', type=open)
parser.add_argument('dest_file', type=argparse.FileType('w', encoding='latin-1'))
parser.add_argument('datapath', type=pathlib.Path);

In [213]:
def hyphenated(string):
    return '-'.join([word[:4] for word in string.casefold().split()])

parser = argparse.ArgumentParser()
_ = parser.add_argument('short_title', type=hyphenated)

In [214]:
parser.parse_args(['"The Tale of Two Cities"'])

Namespace(short_title='"the-tale-of-two-citi')

## 6. choices

功能: 接受有限的几种可选值作为输入, 如果提供的实参不是可接受的值之一就会报错.

In [222]:
parser = argparse.ArgumentParser()
parser.add_argument('move', choices=['rock', 'paper', 'scissors'])
parser.add_argument('--door', type=int, choices=range(1, 4));

In [225]:
parser.parse_args(['rock','--door','2'])

Namespace(door=2, move='rock')

In [ ]:
parser.parse_args(['fire']) # 报错

In [ ]:
parser.parse_args(['rock','--door','4']) # 报错

## 7. required

功能: 把可选的参数变成必需的.

In [230]:
parser = argparse.ArgumentParser()
parser.add_argument('--foo', required=True)
parser.parse_args(['--foo', 'BAR'])

Namespace(foo='BAR')

In [231]:
parser.parse_args([])

usage: ipykernel_launcher.py [-h] --foo FOO
ipykernel_launcher.py: error: the following arguments are required: --foo


SystemExit: 2

## 8. help

功能: 请求帮助时显示包含参数简短描述的字符串.

In [235]:
parser = argparse.ArgumentParser(prog='frobble')
parser.add_argument('--foo', action='store_true',
                    help='foo the bars before frobbling')
parser.add_argument('bar', nargs='+',
                    help='one of the bars to be frobbled')
parser.add_argument('--boo', nargs='?', type=int, default=42,
                    help='the bar to %(prog)s (default: %(default)s)')
parser.parse_args(['-h'])

usage: frobble [-h] [--foo] [--boo [BOO]] bar [bar ...]

positional arguments:
  bar          one of the bars to be frobbled

optional arguments:
  -h, --help   show this help message and exit
  --foo        foo the bars before frobbling
  --boo [BOO]  the bar to frobble (default: 42)


SystemExit: 0

In [239]:
parser.parse_args('1 2 3 --foo'.split())

Namespace(bar=['1', '2', '3'], boo=42, foo=True)

## 9. metavar

功能: 显示帮助信息时指定预期参数的引用名字. 如果不指定, 则位置参数的默认名字为参数名本身, 可选参数的默认名字为自己的大写.

In [245]:
## 不指定metavar
parser = argparse.ArgumentParser()
parser.add_argument('--foo')
parser.add_argument('bar')
parser.print_help()

usage: ipykernel_launcher.py [-h] [--foo FOO] bar

positional arguments:
  bar

optional arguments:
  -h, --help  show this help message and exit
  --foo FOO


In [246]:
## 指定metavar
parser = argparse.ArgumentParser()
parser.add_argument('--foo', metavar='YYY')
parser.add_argument('bar',metavar='XXX')
parser.print_help()

usage: ipykernel_launcher.py [-h] [--foo YYY] XXX

positional arguments:
  XXX

optional arguments:
  -h, --help  show this help message and exit
  --foo YYY


In [252]:
# 如果某个命令行参数需要提供多个实参, 则相应的提供一个元组给metavar
parser = argparse.ArgumentParser(prog='PROG')
parser.add_argument('-x', nargs=2)
parser.add_argument('--foo', nargs=2, metavar=('bar', 'baz'))
parser.print_help()

usage: PROG [-h] [-x X X] [--foo bar baz]

optional arguments:
  -h, --help     show this help message and exit
  -x X X
  --foo bar baz


## 10. dest

功能: 指定 `parse_args()`返回对象的属性名.默认情况下:
- 对于位置参数动作，dest 通常会采用 add_argument() 的第一个参数
- 对于可选参数动作，dest 的值通常取add_argument()的第一个长选项字符串并去掉开头的 -- 字符串来生成 dest 的值。 如果没有提供长选项字符串，采用第一个短选项字符串并去掉开头的 - 字符。

In [255]:
## 位置参数
parser = argparse.ArgumentParser()
parser.add_argument('bar')
parser.parse_args(['XXX'])

Namespace(bar='XXX')

In [259]:
## 可选参数
parser = argparse.ArgumentParser()
parser.add_argument('-f', '--foo-bar', '--foo')
parser.add_argument('-x', '-y');
parser.print_help()

usage: ipykernel_launcher.py [-h] [-f FOO_BAR] [-x X]

optional arguments:
  -h, --help            show this help message and exit
  -f FOO_BAR, --foo-bar FOO_BAR, --foo FOO_BAR
  -x X, -y X


In [257]:
parser.parse_args('-f 1 -x 2'.split())

Namespace(foo_bar='1', x='2')

In [258]:
parser.parse_args('--foo 1 -y 2'.split())

Namespace(foo_bar='1', x='2')

In [260]:
## 自定义
parser = argparse.ArgumentParser()
parser.add_argument('--foo', dest='bar')
parser.parse_args('--foo XXX'.split())

Namespace(bar='XXX')

## parse_args()指定选项值的方式

### 1. 选项和它的值是作为两个单独参数传入

In [261]:
parser = argparse.ArgumentParser(prog='PROG')
parser.add_argument('-x')
parser.add_argument('--foo');

In [262]:
parser.parse_args(['-x', 'X'])

Namespace(foo=None, x='X')

In [263]:
parser.parse_args(['--foo', 'FOO'])

Namespace(foo='FOO', x=None)

### 2. 对于长选项（名称长度超过一个字符的选项)
选项和值也可以作为单个命令行参数传入，使用 = 分隔它们即可

In [264]:
parser.parse_args(['--foo=FOO'])

Namespace(foo='FOO', x=None)

### 3. 对于短选项（长度只有一个字符的选项)
选项和它的值可以拼接在一起:

In [265]:
parser.parse_args(['-xX'])

Namespace(foo=None, x='X')

### 4. 有些短选项可以使用单个 - 前缀来进行合并
如果仅有最后一个选项（或没有任何选项）需要值的话:

In [266]:
parser = argparse.ArgumentParser(prog='PROG')
parser.add_argument('-x', action='store_true')
parser.add_argument('-y', action='store_true')
parser.add_argument('-z');

In [269]:
parser.parse_args(['-xyzZ'])

Namespace(x=True, y=True, z='Z')

## 参数缩写(前缀匹配)

parse_args() 方法 在默认情况下 允许将长选项缩写为前缀，如果缩写无歧义（即前缀与一个特定选项相匹配）的话:

In [284]:
parser = argparse.ArgumentParser(prog='PROG')
parser.add_argument('-bacon')
parser.add_argument('-badger');

In [285]:
parser.parse_args('-bac MMM'.split())

Namespace(bacon='MMM', badger=None)

In [286]:
parser.parse_args('-bad WOOD'.split())

Namespace(bacon=None, badger='WOOD')

In [287]:
parser.parse_args('-ba BA'.split())

usage: PROG [-h] [-bacon BACON] [-badger BADGER]
PROG: error: ambiguous option: -ba could match -bacon, -badger


SystemExit: 2

# LAB

In [15]:
parser = argparse.ArgumentParser(prog='searchdiff.py',
                                    description='Detect and rank source on diffenrence image, require sklearn<0.23.',
                                    formatter_class=argparse.ArgumentDefaultsHelpFormatter)
parser.add_argument('-d', '--difimg',
                    metavar='diff.fits',
                    help='path to diffenrence image.')
parser.add_argument('-s', '--sciimg',
                    metavar='science.fits',
                    help='path to science image.')
parser.add_argument('-t', '--tplimg',
                    metavar='template.fits',
                    help='path to aligned template image.')
parser.add_argument('--fwhm',
                    type=float,
                    help='FWHM in pixel.')
parser.add_argument('--magzp',
                    type=float,
                    metavar='MAG_ZEROPOINT',
                    default=0.0,
                    help='mag zeropoint of difference(science) image.')
parser.add_argument('--gain',
                    type=float,
                    metavar='GAIN',
                    default=0.0,
                    help='detector gain in e-/ADU.')
parser.add_argument('--dthres',
                    type=float,
                    metavar='DETECT_THRESH',
                    default=1.5,
                    help='SExtractor config param')
parser.add_argument('--athres',
                    type=float,
                    metavar='ANALYSIS_THRESH',
                    default=1.5,
                    help='SExtractor config param')
parser.add_argument('--sexpath',
                    default='/usr/bin/sex',
                    metavar='/path/to/sex',
                    help='path to SExtractor')
parser.add_argument('--sexconf',
                    metavar='config.sex',
                    help='path to SExtractor config file, optional.');                 

In [16]:
args = parser.parse_args(['--magzp','26'])

In [19]:
args

Namespace(athres=1.5, difimg=None, dthres=1.5, fwhm=None, gain=0.0, magzp=26.0, sciimg=None, sexconf=None, sexpath='/usr/bin/sex', tplimg=None)